# Ingest Real-Time Stock Data to Iguazio NoSQL and Time-series DB
the following example function ingest real-time stock information from an internet service (Yahoo finance api) into iguazio platform.<br>
everytime the data is updated it updates a NoSQL table with the recent metadata and updates the time-series DB with the new metrics (price and volume)

The same code can run inside a nuclio (serverless) function and be automatically triggered on a predefined schedule (cron) or through HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations (such as the cron schedule), and to deploy functions automatically onto a cluster.

In [11]:
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 
import os

## Environment

copy the local credentials to the nuclio function config (-c option doesn't initialize locally)

In [18]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [19]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [3]:
%%nuclio cmd -c
pip install lxml
pip install yfinance
pip install requests
pip install v3io_frames

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [6]:
# nuclio: start-code

In [12]:
import yfinance as yf
import os
import pandas as pd
import v3io_frames as v3f
import ast
import mlrun.feature_store as fs
import mlrun
import datetime
from mlrun.datastore.targets import ParquetTarget

In [13]:
def update_tickers(context, start, end, interval):
    all_records=[]
    stocks_df = pd.DataFrame()
    for sym in context.stock_syms:
        hist = yf.Ticker(sym).history(start=start, end=end,interval=interval).drop(["Dividends","Stock Splits"],axis=1)
        time = hist.index[len(hist) - 1]
        record = hist.loc[time]
        last = context.last_trade_times.get(sym)
        if last:
            hist = hist[hist.index>last]
        context.logger.info(f'Received {sym} data from yfinance, including {len(hist)} candles ending at {last}')
        # update the stocks table and TSDB metrics in case of new data
        if not last or time > last:
            fs_data = hist.copy().reset_index()
            fs_data["symbol"]=sym
            fs_data["ticker"]=sym
            fs_data.set_index("symbol",inplace=True)
            # ingesting new data to a featureset
            context.logger.info(f"Writing new dataframe with shape {fs_data.shape} to feature store")
#             fs_data["Datetime"] = fs_data["Datetime"].dt.tz_localize('UTC')
            fs_data["Datetime"] = fs_data["Datetime"].dt.tz_convert('UTC')
            fs.ingest(context.stock_info_feature_set, fs_data, infer_options=fs.InferOptions.default())
            
            # update NoSQL table with stock data
            stock = {'symbol': sym, 'price': record['Close'], 'volume': record['Volume'], 'last_updated': time}
            #all_records.append(stock)
            expr = context.expr_template.format(**stock)
            context.logger.debug_with('update expression', symbol=sym, expr=expr)
            context.v3c.execute('kv', context.stocks_kv_table, 'update', args={'key': sym, 'expression': expr})
         
            context.logger.info(f'Updated records from {last} to {time}')
            # update time-series DB with price and volume metrics (use pandas dataframe with a single row, indexed by date)
            context.last_trade_times[sym] = time
            hist['symbol'] = sym
            hist = hist.reset_index()
            hist = hist.set_index(['Datetime', 'symbol'])
            hist = hist.loc[:, ['Close', 'Volume']]
            hist = hist.rename(columns={'Close': 'price', 'Volume': 'volume'})
            stocks_df = stocks_df.append(hist)
            context.logger.info(f'Added records {hist.shape[0]} records for {sym} to history')
        else:
            context.logger.info(f'No update was made, current TS: {last} vs. new data {time}')
    
    # inferring KV to create a scheme
#     context.v3c.execute("kv",table = context.stocks_kv_table, command = "infer")
    
    # write price and volume metrics to the Time-Series DB, add exchange label
    if stocks_df.shape[0]>0:
        stocks_df = stocks_df.sort_index(level=0)
        context.logger.debug_with('writing data to TSDB', stocks=stocks_df)
        stocks_df.to_csv('history.csv')
        context.v3c.write(backend='tsdb', table=context.stocks_tsdb_table, dfs=stocks_df)

In [14]:
def init_context(context):
    context.logger.info("init stocks reader context")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks2-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project = context.PROJECT_NAME)
    # Setup V3IO Client
    setattr(context,"V3IO_FRAMESD", os.getenv("V3IO_FRAMESD",'framesd:8081'))
    client = v3f.Client(context.V3IO_FRAMESD, container=os.getenv('V3IO_CONTAINER', 'users'))
    setattr(context, 'v3c', client)
    
    # Create V3IO Tables and add reference to context
    setattr(context, 'stocks_kv_table', os.getenv('STOCKS_KV_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_kv'))
    setattr(context, 'stocks_tsdb_table', os.getenv('STOCKS_TSDB_TABLE', os.getenv('V3IO_USERNAME') + '/stocks/stocks_tsdb'))
    context.v3c.create(backend='tsdb', table=context.stocks_tsdb_table, rate='1/m', if_exists=1)
    
    # Supply the feature set to ingest data to.
    stocks_info_set = fs.FeatureSet("stocks", entities=[fs.Entity("symbol")],timestamp_key="Datetime")
    stocks_info_set.set_targets(targets=[ParquetTarget(name="stocks",partitioned=True,time_partitioning_granularity="day")]
                                ,with_defaults=False)
    setattr(context,'stock_info_feature_set',stocks_info_set)
    
    # Adding aggregations
    context.stock_info_feature_set.add_aggregation("Opens","Open",["min","max"],["1h"])
    context.stock_info_feature_set.add_aggregation("Volumes","Volume",["min","max"],["1h"])
    
    stocks = os.getenv('STOCK_LIST','GOOGL,MSFT,AMZN,AAPL,INTC')
    if stocks.startswith('['):
        stock_syms = ast.literal_eval(stocks)
    else: 
        stock_syms = stocks.split(',')
    setattr(context, 'stock_syms', stock_syms)
    

    # v3io update expression template 
    expr_template = os.getenv('EXPRESSION_TEMPLATE', "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'")
    setattr(context, 'expr_template', expr_template)

    last_trade_times = {}
    setattr(context, 'last_trade_times', last_trade_times)
    
    # Run first initial data preperation
    start = datetime.datetime.now()-datetime.timedelta(4)
    end = datetime.datetime.now()-datetime.timedelta(2)
    s_df = update_tickers(context, start=start, end=end,interval="1m")

In [15]:
def handler(context,event):
    start = datetime.datetime.now()-datetime.timedelta(3)
    end = datetime.datetime.now()
    stocks_df = update_tickers(context, start=start, end=end,interval="1m")

In [16]:
# nuclio: end-code

## Function invocation
### Local test
the following section simulates nuclio function invocation and will emit the function results

In [17]:
# create a test event and invoke the function locally 
init_context(context)
event = ""
s = handler(context,event)

Python> 2021-06-29 08:18:42,216 [info] init stocks reader context
- GOOGL: No data found for this date range, symbol may be delisted


KeyError: "['Dividends' 'Stock Splits'] not found in axis"

In [10]:
s.index.unique()

Index(['INTC'], dtype='object', name='symbol')

## Deploy to cluster

In [23]:
from mlrun import code_to_function

# Export bare function
fn = code_to_function('read-stocks',
                      handler='handler')
fn.export('01-read-stocks.yaml')

# Set parameters for current deployment
fn.add_trigger('cron', nuclio.triggers.CronTrigger('300s'))
fn.set_envs({'STOCK_LIST': ['GOOG', 'MSFT', 'AMZN', 'AAPL', 'INTC'],
             'V3IO_CONTAINER': 'users' ,
             'STOCKS_TSDB_TABLE': os.getenv('V3IO_USERNAME')  + 'stocks/stocks_tsdb',
             'STOCKS_KV_TABLE': os.getenv('V3IO_USERNAME')  + 'stocks/stocks_kv',
             'EXPRESSION_TEMPLATE': "symbol='{symbol}';price={price};volume={volume};last_updated='{last_updated}'",
             'PROJECT_NAME' : "stocks3-" + os.getenv('V3IO_USERNAME')})
fn.spec.max_replicas = 1

> 2021-06-29 09:07:59,410 [info] function spec saved to path: 01-read-stocks.yaml


In [24]:
addr = fn.deploy(project='stocks3-dani')

> 2021-06-29 09:07:59,418 [info] Starting remote function deploy
2021-06-29 09:07:59  (info) Deploying function
2021-06-29 09:07:59  (info) Building
2021-06-29 09:07:59  (info) Staging files and preparing base images
2021-06-29 09:07:59  (info) Building processor image
2021-06-29 09:08:01  (info) Build complete
2021-06-29 09:08:11  (info) Function deploy complete
> 2021-06-29 09:08:12,481 [info] function deployed, address=default-tenant.app.vmdev31.lab.iguazeng.com:32634


In [25]:
!curl {addr}

E0629 09:08:52.871137645   57785 backup_poller.cc:132]       Run client channel backup poller: {"created":"@1624957732.871010956","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":324,"referenced_errors":[{"created":"@1624957732.871002265","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":954,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
